In [7]:
library(tidyverse)
library(nycflights13)

# Lecture 14: Functions

In this lecture we will learn about functions. We already have plenty of experience calling functions in order to mutate, plot, and explore data. Now we will learn how and when to write our own functions.

## When to write a function

Often when programming we find ourselves repeating the same block of code with minor modifications. Suppose we want to normalize each column of this tibble to be in $[0,1]$:

In [8]:
df = tibble::tibble(
  a = rnorm(10),
  b = rnorm(10),
  c = rnorm(10),
  d = rnorm(10)
) %>% print

# A tibble: 10 x 4
        a      b       c        d
    <dbl>  <dbl>   <dbl>    <dbl>
 1 -0.165  0.398  2.40    0.476  
 2 -0.253 -0.612 -0.0392 -0.710  
 3  0.697  0.341  0.690   0.611  
 4  0.557 -1.13   0.0280 -0.934  
 5 -0.689  1.43  -0.743  -1.25   
 6 -0.707  1.98   0.189   0.291  
 7  0.365 -0.367 -1.80   -0.443  
 8  0.769 -1.04   1.47    0.00111
 9 -0.112  0.570  0.153   0.0743 
10  0.881 -0.135  2.17   -0.590  


To normalize we'll subtract the min from each column and divide by its range:

In [9]:
df$a <- (df$a - min(df$a, na.rm = TRUE)) / 
  (max(df$a, na.rm = TRUE) - min(df$a, na.rm = TRUE))
df$b <- (df$b - min(df$b, na.rm = TRUE)) / 
  (max(df$b, na.rm = TRUE) - min(df$b, na.rm = TRUE))
df$c <- (df$c - min(df$c, na.rm = TRUE)) / 
  (max(df$c, na.rm = TRUE) - min(df$c, na.rm = TRUE))
df$d <- (df$d - min(df$d, na.rm = TRUE)) / 
  (max(df$d, na.rm = TRUE) - min(df$d, na.rm = TRUE))
 
print(df)

# A tibble: 10 x 4
        a      b     c     d
    <dbl>  <dbl> <dbl> <dbl>
 1 0.342  0.491  1     0.927
 2 0.286  0.166  0.420 0.292
 3 0.884  0.473  0.593 1    
 4 0.796  0      0.436 0.171
 5 0.0118 0.824  0.252 0    
 6 0      1      0.474 0.829
 7 0.675  0.245  0     0.435
 8 0.929  0.0274 0.777 0.673
 9 0.375  0.546  0.466 0.712
10 1      0.320  0.946 0.356


This required a bunch of repetitive typing. (Worse, there is an error in the code.) In situations like this we should write a function!

## Anatomy of a function
To write a function we should first think about the inputs and output. A function takes input(s), does something(s) to them, and then returns an output.

What are the input(s) and output of our normalize function?
```{r}
df$a <- (df$a - min(df$a, na.rm = TRUE)) / 
  (max(df$a, na.rm = TRUE) - min(df$a, na.rm = TRUE))
```

We will call our function `rescale01`. The input is the vector we wish to normalize, and the output is the a copy of the vector where each entry is normalized.

In [10]:
rescale01 <- function(x) {
#  ^ function name   ^ function argument (input vector)
    (x - min(x, na.rm = TRUE)) / (max(x, na.rm = TRUE) - min(x, na.rm = TRUE))
#   ^ function output
}

Notice how we have taken our code and converted every instance of `df$a` to `x`, which is the name that we have assigned to our function argument.

Let's test our function on a few examples:

In [11]:
rescale01(c(.01, .2, .3))

[1] 0.0000000 0.6551724 1.0000000

In [12]:
rescale01(c(1,0,3,2))

[1] 0.3333333 0.0000000 1.0000000 0.6666667

Now that we have defined our function, we can replace our code with a nicer looking version:

In [13]:
df$a = rescale01(df$a)
df$b = rescale01(df$b)
df$c = rescale01(df$c)
df$d = rescale01(df$d)

This is considerably simpler, but still has some repetition. Soon we will learn about iteration and ways to cut down further on repetition.

What happens if we pass an infinite value to our function?

In [14]:
x = c(1:10, Inf)
rescale01(x)

 [1]   0   0   0   0   0   0   0   0   0   0 NaN

We have turned up a bug in our function! But since the code now all lives in one place, we can fix the function once rather than having to chase down the bug every place that we copied and pasted the code.

In [15]:
rescale01 = function(x) {
  rng = range(x, na.rm = TRUE, finite = TRUE)
  (x - rng[1]) / (rng[2] - rng[1])
}
rescale01(x)

 [1] 0.0000000 0.1111111 0.2222222 0.3333333 0.4444444 0.5555556 0.6666667
 [8] 0.7777778 0.8888889 1.0000000       Inf

## Conditional execution
Often when writing functions we need to do different things depending on what data is passed in. This is known as *conditional execution*, and is accomplished using the `if/else` construct:
```{r}
if (condition) {
  # code executed when condition is TRUE
} else {
  # code executed when condition is FALSE
}
```

### Example
The *Heaviside step function* is defined as
$$H(x)=\begin{cases}0,&x\le 0\\
1,&\text{otherwise}
\end{cases}.$$
How can we code this as an R function?

In [16]:
H = function(x) {
    if (x <= 0) {
        return(0)
    } else {
        return(1)
    }
}
H(-1:1)

Warning message in if (x <= 0) {:
“the condition has length > 1 and only the first element will be used”

[1] 0

### Example
As we will soon learn, vector entries can be *named*:

In [17]:
(x = c(a=1,b=2,c=3,4,5,d=7))
names(x) %>% as.logical()

a b c     d 
1 2 3 4 5 7 

[1] NA NA NA NA NA NA

### Challenge Problem
Write a function that takes as input a vector and returns a logical vector of the same length. Each entry of the returned vector should equal `TRUE` or `FALSE` according to whether the corresponding entry in the input vector has a name.

In [18]:
has_name = function(x) {
    return(names(x) != "")
} 
x = c(a=1,b=2,c=3,4,5,d=7)
has_name(x)
# has_name(x) -> c(T, T, T, F, F, T)

[1]  TRUE  TRUE  TRUE FALSE FALSE  TRUE

### Challenge problem
Write a function `fizzbuzz(x)` that prints "fizz" if x is divisible by three, and "buzz" otherwise.
```{r}
> fizzbuzz(3)
[1] "fizz"
> fizzbuzz(4)
[2] "buzz"
```

In [19]:
x = 1:10
x %% 3 == 0
ifelse(
    x %% 3 == 0, 
    "fizz", 
    "buzz"
)

 [1] FALSE FALSE  TRUE FALSE FALSE  TRUE FALSE FALSE  TRUE FALSE

 [1] "buzz" "buzz" "fizz" "buzz" "buzz" "fizz" "buzz" "buzz" "fizz" "buzz"

### Review Problem
What do the following functions do?
```{r}
f1 <- function(string, prefix) {
  substr(string, 1, nchar(prefix)) == prefix
}

f2 <- function(x) {
  if (length(x) <= 1) return(NULL)
  x[-length(x)]
}
```

### Conditions
The `condition` part of the `if` statement must evaluate to either a single `TRUE` or `FALSE`. If it does not, you will get a warning:

In [52]:
if (c(T, F)) { 1 } else { }
ifelse(
    1:10 > 5,
    "gt5",
    "lt5"
)

Warning message in if (c(T, F)) {:
“the condition has length > 1 and only the first element will be used”

[1] 1

 [1] "lt5" "lt5" "lt5" "lt5" "lt5" "gt5" "gt5" "gt5" "gt5" "gt5"

(Why?) Similarly, a condition of `NA` will generate an error:
```{r}
> if (NA) { 1 }
Error in if (NA) {: missing value where TRUE/FALSE needed
Traceback:
```

#### Logical operators
Often you will need to combine multiple logical conditions in an `if` statement. To do this we have the `&&` and `||` operators, which take the logical `and` and `or`, respectively, of several logical conditions:

In [54]:
TRUE && FALSE && TRUE

c(T, T, F) && c(T, F, T)  # Vectorized version. single &

[1] FALSE

[1] TRUE

In [22]:
FALSE || TRUE || FALSE

[1] TRUE

There is a subtle but important difference betwen the single and double versions of these operators. The single `&` performs entrywise `AND` over logical vectors:

In [23]:
c(T, T, F) & c(F, T, F)

[1] FALSE  TRUE FALSE

In contrast, the double ampersand `&&` returns `F` as soon as it encounters a value of `F`:

In [24]:
c(T, T, T) && c(F, T, F)

[1] FALSE

It only returns `T` if it gets to the end of a vector without finding any `F` values:

In [25]:
c(T, T, T) && c(T, T, T)

[1] TRUE

This is known as "short-circuiting": R can stop evaluating as soon as it hits *one* false value, since this will cause the `&` to return false:

In [26]:
f = function() { print("f called"); F }
g = function() { print("g called"); T }
f() && g()

[1] "f called"


[1] FALSE

The or operator works similarly:

In [27]:
g() || f()

[1] "g called"


[1] TRUE

#### Testing for equality
Be careful when testing for equality in conditionals. The `==` operator will return a *vector* of logicals. If you want to make sure that any/all entries of a vector are `TRUE`, use the `any()` or `all()` functions:

In [58]:
v1 = c(1, 2, 3)
v2 = c(1, 1, 2)
if (v1 == v2) { print("Wrong!") }
if (all(v1 == v2)) { print("All!") }
if (any(v1 == v2)) { print("Any!") }

[1] TRUE


Warning message in if (v1 == v2) {:
“the condition has length > 1 and only the first element will be used”

[1] "Wrong!"
[1] "Any!"


Also be wary of testing floating point numbers for equality:

In [65]:
2 == 3

[1] FALSE

If you need to do this, use the `near()` function instead:

In [69]:
abs(2 - sqrt(2) ^ 2) < 1e-12

[1] TRUE

### Multiple conditions
Sometimes you will want to check multiple conditions using an `if` statement. For example, let's define the function $$\operatorname{sgn}(x) = \begin{cases}-1,&x<0\\0,&x=0\\1,&x>0.\end{cases}$$

In [73]:
sgn = function(x) {
    if (x < 0) {
        return(-1)
    } else if (x == 0) {
        return(0)
    }
    return(1)
}
sgn(0)

[1] 0

The general form is
```{r}
if (this) {
  # do that
} else if (that) {
  # do something else
} else {
  # 
}
```

If you find yourself chaining together a long string of `if/else if/else if/else if/.../else` statements, chances are there is an easier way. For example, say we have a continuous variable `temp` that we want to convert to a factor. One way is using lots of `if/else`:
```{r}
if (temp <= 0) {
  "freezing"
} else if (temp <= 10) {
  "cold"
} else if (temp <= 20) {
  "cool"
} else if (temp <= 30) {
  "warm"
} else {
  "hot"
}
```

Alternative, we could use the `cut` function:
```{r}
cut(temp, c(0, 10, 20, 30), 
    c('freezing', 'cold', 'cool', 'warm', 'hot'))
```

### Brackets
Both `function` and `if` are usually called using the curly bracket delimiters `{` and `}`. For one-line statements, the brackets are optional:

In [75]:
if (TRUE) { 
    print("A1") 
} else { 
    print("B") 
}

if (TRUE) print("A2") else print("B")

[1] "A1"
[1] "A2"


You should almost always use the curly braces. One exception is for very brief, unnamed functions. We'll see some examples of this next week when we study map/reduce computations.

## Function arguments
Functions can take multiple arguments. Generally they fall into one of two categories:
* *Data* to be processed by the function, and
* *Options*, which affect how the data gets processed.

```{r}
mean(x, na.rm=TRUE)
log(x, base=y)
str_c(..., sep=" ")
```
What is/are the data? What are the options?

### Rules for function arguments
Generally:
1. The *data* parameters should come first; and
2. The *options* should come second, and have sensible defaults.

Default parameter values are specified by the `option=default` notation:

In [33]:
mean_ci <- function(x, conf=0.95) {
  se <- sd(x) / sqrt(length(x))
  alpha <- 1 - conf
  mean(x) + se * qnorm(c(alpha / 2, 1 - alpha / 2))
}

When you call a function, you can omit the values of the default arguments. If overriding the default, you should specify the parameter you are overriding and then input the overridden value with an ` = ` in between:
```{r}
mean_ci(c(1, 2, 3, 4))  # standard
mean_ci(c(1, 2, 3, 4), conf = .99)  # yes
mean_ci(c(1, 2, 3, 4), conf=.99)  # ok (Pythonic)
mean_ci(c(1, 2, 3, 4), .99)  # no
```

### Validation
When writing functions it's a good idea to *validate* the input -- that is, make sure it matches your assumptions about what is being passed to the function. Consider the following function which returns the weighted average of a vector:

In [77]:
w_mean = function(x, w) {
    (x * w) / sum(w)
}

This function relies implicitly on the fact that the weight vector `w` is the same length as the input vector `x`. If it's not, you'll get a warning and unexpected behavior.

In [80]:
w_mean(c(1,2,3), w=c(1, 2))

ERROR: Error: length(w) == length(x) is not TRUE


It's best to make the assumption of equal length explicit by checking it:

In [79]:
w_mean = function(x, w) {
    stopifnot(length(w) == length(x))
    (x * w) / sum(w)
}

Now:
```{r}
> w_mean(c(1,2,3), w=c(1, 2))
Error: length(w) == length(x) is not TRUE
Traceback:

1. w_mean(c(1, 2, 3), w = c(1, 2))
2. stopifnot(length(w) == length(x))   # at line 2 of file <text>
3. stop(msg, call. = FALSE, domain = NA)
```

Adding comments is another good way to make sure that you don't encounter unexpected situations in your functions:

In [38]:
w_mean = function(x, w) {
    # Return the average of `x` weighted by weight vector `w`
    stopifnot(length(w) == length(x))
    (x * w) / sum(w)
}

###  Dot-dot-dot (`…`)
Some functions are designed to take a variable number of inputs. We saw this for example with the `str_c` function:

In [39]:
stringr::str_c("a", "b")
stringr::str_c("a", "b", "c", "d")

[1] "ab"

[1] "abcd"

To construct a function that takes a variable number of arguments we use the `...` notation:
```{r}
f = function(...) {
    <do something with variable arguments>
}
```

One thing you can do with the `...` is pass it to another function:

In [40]:
commas <- function(...) stringr::str_c(..., collapse = ", ")
commas(letters[1:10])

[1] "a, b, c, d, e, f, g, h, i, j"

You can also access individual arguments in `...` using the `list(...)` notation. We'll learn more about lists in the next lecture.

## Return values
Thus far we have relied on the default behavior of R, which is to return the last value in the function:

In [41]:
f = function() {
    1
    2
    3  # this will be returned
}
f()

[1] 3

In more complicated functions you'll need to manually return values using the `return()` function:
```{r}
complicated_function <- function(x, y, z) {
  if (length(x) == 0 || length(y) == 0) {
      return(0)  # this immediately returns and halts the function.
  }
  # Complicated code here
}
```

### Pipeable functions
We've seen a lot of uses of the pipe operator `%>%`. As you become more advanced, you may find it useful to create your own functions which can be used in data pipelines. 

#### Transformations
For pipeable functions that transform a data frame, simply return the altered version of the data frame. For example:

In [81]:
first_row <- function(df) {
    df %>% slice(1)
} 
tibble(x=c(1,2,3), y=c("a","b","c")) %>% first_row

  x y
1 1 a

### Challenge problem
Define a function `drop_even()` which drops all the even-numbered rows from a data frame:
```{r}
> tibble(x=c(1,2,3), y=c("a","b","c")) %>% drop_even
# A tibble: 2 x 2
      x y    
  <int> <chr>
1     1 a    
2     3 c    
```

In [90]:
drop_even = function(df) {
    n <- nrow(df)
    odd_rows <- seq(from = 1, to = n, 2)
    df %>% slice(odd_rows)
}
# 
drop_even(mpg) %>% head

  manufacturer model      displ year cyl trans      drv cty hwy fl class  
1 audi         a4         1.8   1999 4   auto(l5)   f   18  29  p  compact
2 audi         a4         2.0   2008 4   manual(m6) f   20  31  p  compact
3 audi         a4         2.8   1999 6   auto(l5)   f   16  26  p  compact
4 audi         a4         3.1   2008 6   auto(av)   f   18  27  p  compact
5 audi         a4 quattro 1.8   1999 4   auto(l5)   4   16  25  p  compact
6 audi         a4 quattro 2.0   2008 4   auto(s6)   4   19  27  p  compact

#### Side effects
Some functions have *side effects* but don't modify the original data frame. For example, consider the following function which counts how many missing values are present in a data frame:

In [44]:
show_missings = function(df) {
  n <- sum(is.na(df))
  cat("Missing values: ", n, "\n", sep = "")
  df  # note return value
}

This function works but has the undesirable effect of printing the whole data frame when it returns:

In [45]:
show_missings(mpg)

Missing values: 0


    manufacturer model              displ year cyl trans      drv cty hwy fl
1   audi         a4                 1.8   1999 4   auto(l5)   f   18  29  p 
2   audi         a4                 1.8   1999 4   manual(m5) f   21  29  p 
3   audi         a4                 2.0   2008 4   manual(m6) f   20  31  p 
4   audi         a4                 2.0   2008 4   auto(av)   f   21  30  p 
5   audi         a4                 2.8   1999 6   auto(l5)   f   16  26  p 
6   audi         a4                 2.8   1999 6   manual(m5) f   18  26  p 
7   audi         a4                 3.1   2008 6   auto(av)   f   18  27  p 
8   audi         a4 quattro         1.8   1999 4   manual(m5) 4   18  26  p 
9   audi         a4 quattro         1.8   1999 4   auto(l5)   4   16  25  p 
10  audi         a4 quattro         2.0   2008 4   manual(m6) 4   20  28  p 
11  audi         a4 quattro         2.0   2008 4   auto(s6)   4   19  27  p 
12  audi         a4 quattro         2.8   1999 6   auto(l5)   4   15  25  p 

To correct this tidyverse has the `invisible()` function:

In [46]:
show_missings = function(df) {
  n <- sum(is.na(df))
  cat("Missing values: ", n, "\n", sep = "")
  invisible(df)  # return will not print out
}

Now we can run the command interactively, and also use it in pipelines:

In [47]:
show_missings(flights)

Missing values: 46595


In [48]:
flights %>% filter(month < 5) %>% show_missings

Warning message:
“package ‘bindrcpp’ was built under R version 3.4.4”

Missing values: 18387


## Environments
The environment is, roughly, the set of variables and data defined in your R session. The default environment is called the "global environment":

In [95]:
environment()$drop_even 

function(df) {
    n <- nrow(df)
    odd_rows <- seq(from = 1, to = n, 2)
    df %>% slice(odd_rows)
}

A function depends on the environment in which it was defined. In particular, if you reference a variable inside of a function, which is not *defined* in that function, R will look for it in the enclosing environment.

In [96]:
f = function(x) {
    x + y
}
x = 1:3
y = 3
f(x)

[1] 4 5 6

In [97]:
y = 10
f(x)

[1] 11 12 13

### Challenge problem
Write a function `howmany()` which prints the number of times that it has been called:
```
> howmany()
[1] 1
> howmany()
[1] 2
> howmany()
[1] 3
```

In [130]:
i <- 0
howmany <- function() {
    i <<- i + 1
    return(i)
}

In [146]:
howmany()

[1] 16


Harder: write a function `same(x)` which prints "yes!" if `x` is the same value that was passed into `same()` on the previous call, and "no!" otherwise. (`same()` always prints "no!" to start):
```{r}
> same(1)
[1] "no!"
> same(1)
[1] "yes!"
> same(1)
[1] "yes!"
> same("hello")
[1] "no!"
```